In [1]:
import tensorflow as tf

# implementation of PA % K: https://github.com/tuslkkk/tadpak

aecModel = None
eifModel = None

def autoencoderPredict(data, threshold):
  reconstructions = aecModel(data)
  loss = tf.keras.losses.mae(reconstructions, data)
  return tf.math.less(loss, threshold), loss

def eifPredict(data, aecLoss):
    data.append(aecLoss)
    return eifModel.predict(data)

def isAnomaly(data):
    threshold = None
    # need autoencoder to return boolean isAnomaly and list of features
    isAnomaly, aecFeatures = autoencoderPredict(data, threshold)

    if not isAnomaly:
        return False

    else:
        return eifPredict(data, aecFeatures)


In [2]:
"""
Questions:
How do we obtain "windows" from our datasets?
What should k be for our data? Look at paper
How do we obtain our threshold? It would be a line between the average True point and average False point
"""

def pak(anomaly_segment_list, ground_truth, threshold,  k):
    allAboveThreshold = True

    for item in anomaly_segment_list:
        if item <= threshold:
            allAboveThreshold = False

    if allAboveThreshold:
        return True

    numCorrectlyDetected = 0

    for i in range(len(anomaly_segment_list)):
        if anomaly_segment_list[i] == ground_truth[i]:
            numCorrectlyDetected += 1

    return numCorrectlyDetected / len(anomaly_segment_list) > k